# Hidden Markov Models for Power Consumption (Jan–Feb 2017)

- Dataset: `z1_hr_mean.csv` with hourly means (`mean_power`) from 2017-01-01 to 2017-02-28
- Task: Fit HMMs with different numbers of states; interpret, check fit, and recommend a model
- Methods: HMMs with Gaussian emissions; EM (Baum–Welch), global decoding (Viterbi), pseudo-residual diagnostics; model selection via AIC/BIC

References consulted: `AMT-lecture.md`, `HMM-Tutorial.md`.


In [ ]:
# Setup ----
# Use only R

suppressPackageStartupMessages({
  library(readr)
  library(dplyr)
  library(lubridate)
  library(ggplot2)
  library(tidyr)
})

# HMM packages. Prefer depmixS4 for stability; fallback to HiddenMarkov
have_hiddenmarkov <- requireNamespace("HiddenMarkov", quietly = TRUE)
have_depmix <- requireNamespace("depmixS4", quietly = TRUE)
have_mhsmm <- requireNamespace("mhsmm", quietly = TRUE)

if (!have_hiddenmarkov && !have_depmix && !have_mhsmm) {
  stop("No HMM package available (HiddenMarkov, depmixS4, mhsmm). Please install one.")
}

read_power_data <- function(path_csv) {
  df <- read_csv(path_csv, show_col_types = FALSE)
  # Expect columns: day_in_year (DD/MM/YYYY), hr (0-23), mean_power, sqe
  df <- df %>%
    mutate(
      date = dmy(day_in_year),
      datetime = as.POSIXct(date + hours(hr), tz = "UTC")
    ) %>%
    arrange(datetime) %>%
    filter(!is.na(mean_power))
  return(df)
}

power <- read_power_data("/home/bekind/Documents/purczec1-brr/z1_hr_mean.csv")

nrow(power); summary(power$mean_power)

# Plot series
p_series <- ggplot(power, aes(x=datetime, y=mean_power)) +
  geom_line(color="#2c7fb8", linewidth=0.4) +
  labs(x="Time", y="Mean power (kWh)", title="Hourly mean power (Jan–Feb 2017)") +
  theme_minimal(base_size = 12)
print(p_series)

# Basic ACF to see serial dependence
acf_vals <- acf(power$mean_power, plot=FALSE, lag.max = 72)
plot(acf_vals, main="ACF of mean_power")

# Save quicklook figures
out_dir <- "/home/bekind/Documents/purczec1-brr/outputs"
dir.create(out_dir, showWarnings = FALSE, recursive = TRUE)
try(ggsave(file.path(out_dir, "series.png"), p_series, width=11, height=4, dpi=120))
png(file.path(out_dir, "acf_mean_power.png"), width=1000, height=600)
plot(acf_vals, main="ACF of mean_power")
dev.off()


In [ ]:
# HMM helpers ----

# Fit Gaussian-emission HMM via HiddenMarkov (preferred per lecture) or depmixS4 (fallback)
fit_hmm_gaussian <- function(y, m, seed=123) {
  set.seed(seed)
  y <- as.numeric(y)
  # Special case: one-state model as simple Gaussian
  if (m == 1) {
    mu <- mean(y, na.rm=TRUE)
    sdv <- stats::sd(y, na.rm=TRUE)
    ll <- sum(dnorm(y, mean=mu, sd=sdv, log=TRUE))
    k <- 2
    n <- length(y)
    aic <- 2*k - 2*ll
    bic <- log(n)*k - 2*ll
    comp <- cbind(mean=mu, sd=sdv)
    return(list(pkg="one", model=NULL, logLik=ll, k=k, AIC=aic, BIC=bic, posterior=NULL, viterbi=rep(1L, n), comp=comp))
  }
  # Try HiddenMarkov first, fallback to depmixS4 on error or if unavailable
  if (have_hiddenmarkov) {
    res <- tryCatch({
      library(HiddenMarkov)
      mu0 <- as.numeric(quantile(y, probs = seq(0.2, 0.8, length.out = m)))
      sd0 <- rep(stats::sd(y, na.rm=TRUE)/2, m)
      Pi0 <- rep(1/m, m)
      A0 <- matrix(1/m, m, m)
      model <- dthmm(x = y, Pi = A0, delta = Pi0, distn = "norm", pm = list(mean=mu0, sd=sd0))
      bw <- BaumWelch(model, control = bwcontrol(maxiter = 1000, posdiff = TRUE, tol=1e-6))
      ll <- -bw$negloglik
      vit <- Viterbi(bw$ptm)
      k <- (m*(m-1)) + m + m + (m-1)
      n <- length(y)
      aic <- 2*k - 2*ll
      bic <- log(n)*k - 2*ll
      comp <- cbind(mean=bw$ptm$pm$mean, sd=bw$ptm$pm$sd)
      list(pkg="HiddenMarkov", model=bw, logLik=ll, k=k, AIC=aic, BIC=bic, posterior=NULL, viterbi=vit, comp=comp)
    }, error=function(e) NULL)
    if (!is.null(res)) return(res)
  }
  if (have_depmix) {
    library(depmixS4)
    mod <- depmix(response = y ~ 1, data = data.frame(y=y), nstates = m, family = gaussian())
    fm <- fit(mod, verbose = FALSE)
    ll <- as.numeric(logLik(fm))
    # robust parameter count
    k <- tryCatch({
      if ("npar" %in% getNamespaceExports("depmixS4")) depmixS4::npar(fm) else as.integer(attr(logLik(fm), "df"))
    }, error=function(e) as.integer(attr(logLik(fm), "df")))
    aic <- AIC(fm); bic <- BIC(fm)
    post <- posterior(fm)
    vpath <- depmixS4::viterbi(fm)
    # Robustly extract state sequence from viterbi result
    stateVec <- tryCatch({
      if (is.data.frame(vpath)) {
        sc <- grep("state", names(vpath), ignore.case = TRUE, value = TRUE)
        as.integer(vpath[[sc[1]]])
      } else if (inherits(vpath, "viterbi")) {
        as.integer(vpath@states)
      } else if (is.vector(vpath)) {
        as.integer(vpath)
      } else {
        rep(NA_integer_, nrow(post))
      }
    }, error=function(e) rep(NA_integer_, nrow(post)))
    # Estimate state means/sds empirically from decoded states
    comp <- lapply(1:m, function(j) {
      idx <- which(stateVec == j)
      mu <- if (length(idx) > 0) mean(y[idx]) else mean(y)
      sdv <- if (length(idx) > 1) stats::sd(y[idx]) else stats::sd(y)
      c(mean=unname(mu), sd=unname(sdv))
    })
    comp <- do.call(rbind, comp)
    return(list(pkg="depmixS4", model=fm, logLik=ll, k=k, AIC=aic, BIC=bic, posterior=post, viterbi=stateVec, comp=comp))
  }
  stop("No suitable HMM backend available.")
}

compute_residuals <- function(y, fit) {
  # Forecast pseudo-residuals using 1-step predictive CDFs if supported; here we approximate using decoded states
  y <- as.numeric(y)
  m <- nrow(fit$comp)
  if (fit$pkg == "depmixS4") {
    path <- as.integer(fit$viterbi)
    mu <- fit$comp[,"mean"]; sd <- fit$comp[,"sd"]
    Fyt <- pnorm(y, mean = mu[path], sd = sd[path])
    Fyt <- pmin(pmax(Fyt, 1e-12), 1-1e-12)
    return(qnorm(Fyt))
  } else if (fit$pkg == "HiddenMarkov") {
    u <- fit$model$ptm$u
    mu <- fit$comp[,"mean"]; sd <- fit$comp[,"sd"]
    z <- numeric(length(y))
    for (t in seq_along(y)) {
      w <- as.numeric(u[t,])
      Fyt <- sum(w * pnorm(y[t], mean=mu, sd=sd))
      Fyt <- pmin(pmax(Fyt, 1e-12), 1-1e-12)
      z[t] <- qnorm(Fyt)
    }
    return(z)
  } else if (fit$pkg == "one") {
    mu <- fit$comp[,"mean"]; sd <- fit$comp[,"sd"]
    Fyt <- pnorm(y, mean=mu, sd=sd)
    Fyt <- pmin(pmax(Fyt, 1e-12), 1-1e-12)
    return(qnorm(Fyt))
  } else {
    return(rep(NA_real_, length(y)))
  }
}

fit_grid <- function(y, states_vec = 1:6, seed=123) {
  res <- lapply(states_vec, function(m) {
    fit <- fit_hmm_gaussian(y, m=m, seed=seed)
    data.frame(m=m, logLik=fit$logLik, k=fit$k, AIC=fit$AIC, BIC=fit$BIC)
  })
  do.call(rbind, res)
}



In [ ]:
# Model selection over number of states ----

set.seed(123)
ms <- 1:6
sel <- fit_grid(power$mean_power, states_vec = ms)
print(sel)

sel_long <- sel %>% pivot_longer(cols=c(AIC,BIC), names_to = "crit", values_to = "value")

ggplot(sel_long, aes(x=m, y=value, color=crit)) +
  geom_line() + geom_point() +
  scale_x_continuous(breaks = ms) +
  labs(x="Number of states m", y="Criterion", title="Model selection (AIC/BIC) vs number of states") +
  theme_minimal(base_size = 12)

best_aic_m <- sel$m[which.min(sel$AIC)]
best_bic_m <- sel$m[which.min(sel$BIC)]
cat("Best by AIC:", best_aic_m, "\n")
cat("Best by BIC:", best_bic_m, "\n")


In [ ]:
# Fit the selected models and inspect ----

m_candidates <- sort(unique(c(best_aic_m, best_bic_m, 2, 3, 4)))
fits <- lapply(m_candidates, function(m) fit_hmm_gaussian(power$mean_power, m=m, seed=123))
names(fits) <- paste0("m", m_candidates)

# Summaries of state means/sds
state_summaries <- lapply(seq_along(fits), function(i) {
  m <- m_candidates[i]
  comp <- as.data.frame(fits[[i]]$comp)
  comp$state <- seq_len(nrow(comp))
  comp$m <- m
  comp
}) %>% dplyr::bind_rows() %>% dplyr::select(m, state, mean, sd)

print(state_summaries)

# Plot decoded means over time for one or two candidate models
plot_decoded <- function(fit, datetimes, y) {
  if (fit$pkg == "depmixS4") {
    path <- fit$viterbi
  } else {
    path <- fit$viterbi
  }
  df <- data.frame(datetime=datetimes, y=y, state=as.integer(path))
  mu <- fit$comp[,"mean"]
  df$state_mean <- mu[df$state]
  ggplot(df, aes(x=datetime)) +
    geom_line(aes(y=y), color="#2c7fb8", linewidth=0.3) +
    geom_step(aes(y=state_mean), color="#d95f0e", linewidth=0.6, alpha=0.8) +
    labs(x="Time", y="Mean power", title=paste0("Decoded state means (m=", nrow(fit$comp), ")")) +
    theme_minimal(base_size = 12)
}

for (nm in names(fits)) {
  print(plot_decoded(fits[[nm]], power$datetime, power$mean_power))
}



In [ ]:
# Pseudo-residual diagnostics ----

diagnostics_plot <- function(z, title_prefix) {
  df <- data.frame(t = seq_along(z), z=z)
  p1 <- ggplot(df, aes(x=t, y=z)) + geom_line(color="#636363", linewidth=0.3) +
    geom_hline(yintercept = 0, linetype="dashed", color="grey50") +
    labs(x="t", y="z_t", title=paste0(title_prefix, ": residual series")) + theme_minimal(12)
  p2 <- ggplot(df, aes(sample=z)) + stat_qq() + stat_qq_line() +
    labs(title=paste0(title_prefix, ": Normal Q-Q")) + theme_minimal(12)
  p3 <- ggplot(df, aes(x=z)) + geom_histogram(bins=40, fill="#9ecae1", color="#2171b5") +
    labs(x="z_t", title=paste0(title_prefix, ": histogram")) + theme_minimal(12)
  p4 <- {
    ac <- acf(z, plot=FALSE, lag.max=72)
    plot(ac, main=paste0(title_prefix, ": residual ACF"))
  }
  pval_lb <- Box.test(z, type="Ljung-Box", lag=24, fitdf=0)$p.value
  message(title_prefix, ": Ljung-Box p-value (lag 24) = ", signif(pval_lb, 4))
  list(p1=p1, p2=p2, p3=p3, pval=pval_lb)
}

res_summary <- list()
for (nm in names(fits)) {
  fit <- fits[[nm]]
  z <- compute_residuals(power$mean_power, fit)
  res_summary[[nm]] <- z
  plots <- diagnostics_plot(z, paste0("m=", nrow(fit$comp)))
  print(plots$p1); print(plots$p2); print(plots$p3)
}



In [ ]:
# Interpretation and recommendation ----

# Choose model by BIC primarily (more parsimonious), fallback to AIC if close
chosen_m <- best_bic_m
fit_chosen <- fits[[paste0("m", chosen_m)]]
cat("Chosen model (by BIC): m=", chosen_m, "\n")

# Report state means and sds
comp <- as.data.frame(fit_chosen$comp)
comp$state <- seq_len(nrow(comp))
print(comp)

# Final figure for the chosen model
print(plot_decoded(fit_chosen, power$datetime, power$mean_power))

# Save selection table to CSV for report
write_csv(sel, "/home/bekind/Documents/purczec1-brr/hmm_model_selection.csv")



In [ ]:
# Additional interpretation: state occupancy and distributions ----

get_viterbi_path <- function(fit) {
  if (fit$pkg == "depmixS4") return(as.integer(fit$viterbi))
  if (fit$pkg == "HiddenMarkov") return(as.integer(fit$viterbi))
  if (fit$pkg == "one") return(rep(1L, nrow(fit$comp)))
  rep(NA_integer_, length(power$mean_power))
}

vpath <- get_viterbi_path(fit_chosen)
occ <- as.data.frame(prop.table(table(vpath)))
colnames(occ) <- c("state", "prop")
occ$state <- as.integer(as.character(occ$state))
comp2 <- comp %>% dplyr::select(state, mean, sd) %>% dplyr::left_join(occ, by="state") %>% dplyr::arrange(mean)
print(comp2)

# Density of mean_power by decoded state
plot_density <- {
  df <- data.frame(mean_power = power$mean_power, state = factor(vpath))
  ggplot(df, aes(x=mean_power, fill=state)) +
    geom_density(alpha=0.35) +
    labs(x="Mean power", y="Density", title=paste0("Distributions by decoded states (m=", chosen_m, ")")) +
    theme_minimal(12)
}
print(plot_density)

# Save selection and summaries
write_csv(comp2, "/home/bekind/Documents/purczec1-brr/chosen_model_state_summary.csv")


In [ ]:
# Save key figures ----

# 1) Model selection plot was printed inline; recreate and save
sel_long <- sel %>% tidyr::pivot_longer(cols=c(AIC,BIC), names_to = "crit", values_to = "value")
p_sel <- ggplot(sel_long, aes(x=m, y=value, color=crit)) +
  geom_line() + geom_point() + scale_x_continuous(breaks = ms) +
  labs(x="Number of states m", y="Criterion", title="Model selection (AIC/BIC)") + theme_minimal(12)

dir.create(out_dir, showWarnings = FALSE, recursive = TRUE)
try(ggsave(file.path(out_dir, "model_selection.png"), p_sel, width=8, height=4, dpi=130))

# 2) Decoded states plot for chosen model
p_dec <- plot_decoded(fit_chosen, power$datetime, power$mean_power)
try(ggsave(file.path(out_dir, "decoded_states.png"), p_dec, width=11, height=4, dpi=130))

# 3) Residual diagnostics for chosen model
z_chosen <- compute_residuals(power$mean_power, fit_chosen)
# time series
p_r_ts <- ggplot(data.frame(t=seq_along(z_chosen), z=z_chosen), aes(x=t, y=z)) +
  geom_line(color="#636363", linewidth=0.3) + geom_hline(yintercept = 0, linetype="dashed", color="grey50") +
  labs(x="t", y="z_t", title="Residual series (chosen model)") + theme_minimal(12)
try(ggsave(file.path(out_dir, "resid_series.png"), p_r_ts, width=8, height=3, dpi=130))
# qq
p_r_qq <- ggplot(data.frame(z=z_chosen), aes(sample=z)) + stat_qq() + stat_qq_line() +
  labs(title="Residual QQ (chosen model)") + theme_minimal(12)
try(ggsave(file.path(out_dir, "resid_qq.png"), p_r_qq, width=5, height=4, dpi=130))
# hist
p_r_hist <- ggplot(data.frame(z=z_chosen), aes(x=z)) + geom_histogram(bins=40, fill="#9ecae1", color="#2171b5") +
  labs(x="z_t", title="Residual histogram (chosen model)") + theme_minimal(12)
try(ggsave(file.path(out_dir, "resid_hist.png"), p_r_hist, width=6, height=4, dpi=130))
# acf image via base
png(file.path(out_dir, "resid_acf.png"), width=900, height=550)
plot(acf(z_chosen, plot=FALSE, lag.max=72), main="Residual ACF (chosen model)")
dev.off()

# 4) State densities plot
try(ggsave(file.path(out_dir, "state_density.png"), plot_density, width=8, height=4, dpi=130))
